In [4]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, insert
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy import Column, Integer, String, DateTime
from dotenv import load_dotenv
import os
import kagglehub
import shutil

In [5]:

dataset_id = "oktayrdeki/traffic-accidents"

target_dir = "../data"

path = kagglehub.dataset_download(dataset_id)
print("Descargado en:", path)

for filename in os.listdir(path):
    source = os.path.join(path, filename)
    destination = os.path.join(target_dir, filename)
    shutil.copy(source, destination)

print("Archivos ahora están en:", os.listdir(target_dir))


Descargado en: /home/kevin/.cache/kagglehub/datasets/oktayrdeki/traffic-accidents/versions/1
Archivos ahora están en: []


In [16]:
csv = ("../data/traffic_accidents.csv")
df = pd.read_csv(csv , delimiter=",")

df = df.where(pd.notna(df), None)

df.head()

,crash_date,traffic_control_device,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,crash_type,...,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,crash_hour,crash_day_of_week,crash_month
0,07/29/2023 01:00:00 PM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,13,7,7
1,08/13/2023 12:11:00 AM,TRAFFIC SIGNAL,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0,1,8
2,12/09/2021 10:30:00 AM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,REAR END,T-INTERSECTION,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,10,5,12
3,08/09/2023 07:55:00 PM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,...,NONINCAPACITATING INJURY,5.0,0.0,0.0,5.0,0.0,0.0,19,4,8
4,08/19/2023 02:55:00 PM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,REAR END,T-INTERSECTION,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,14,7,8


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209306 entries, 0 to 209305
Data columns (total 24 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   crash_date                     209306 non-null  object 
 1   traffic_control_device         209306 non-null  object 
 2   weather_condition              209306 non-null  object 
 3   lighting_condition             209306 non-null  object 
 4   first_crash_type               209306 non-null  object 
 5   trafficway_type                209306 non-null  object 
 6   alignment                      209306 non-null  object 
 7   roadway_surface_cond           209306 non-null  object 
 8   road_defect                    209306 non-null  object 
 9   crash_type                     209306 non-null  object 
 10  intersection_related_i         209306 non-null  object 
 11  damage                         209306 non-null  object 
 12  prim_contributory_cause       

In [18]:

df['crash_date'] = pd.to_datetime(df['crash_date'], format='%m/%d/%Y %I:%M:%S %p')

In [19]:
cols_to_int = [
    "injuries_total", "injuries_fatal", "injuries_incapacitating",
    "injuries_non_incapacitating", "injuries_reported_not_evident",
    "injuries_no_indication"
]
for col in cols_to_int:
    df[col] = df[col].astype("Int64")

In [20]:
df.head()

,crash_date,traffic_control_device,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,crash_type,...,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,crash_hour,crash_day_of_week,crash_month
0,2023-07-29 13:00:00,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0,0,0,0,0,3,13,7,7
1,2023-08-13 00:11:00,TRAFFIC SIGNAL,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0,0,0,0,0,2,0,1,8
2,2021-12-09 10:30:00,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,REAR END,T-INTERSECTION,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0,0,0,0,0,3,10,5,12
3,2023-08-09 19:55:00,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,...,NONINCAPACITATING INJURY,5,0,0,5,0,0,19,4,8
4,2023-08-19 14:55:00,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,REAR END,T-INTERSECTION,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0,0,0,0,0,3,14,7,8


In [21]:
print(df.dtypes)


crash_date                       datetime64[ns]
traffic_control_device                   object
weather_condition                        object
lighting_condition                       object
first_crash_type                         object
trafficway_type                          object
alignment                                object
roadway_surface_cond                     object
road_defect                              object
crash_type                               object
intersection_related_i                   object
damage                                   object
prim_contributory_cause                  object
num_units                                 int64
most_severe_injury                       object
injuries_total                            Int64
injuries_fatal                            Int64
injuries_incapacitating                   Int64
injuries_non_incapacitating               Int64
injuries_reported_not_evident             Int64
injuries_no_indication                  

In [ ]:
load_dotenv()

USER = os.getenv("DB_USER")
PASSWORD = os.getenv("DB_PASSWORD")
HOST = os.getenv("DB_HOST")
DATABASE = os.getenv("DB_RAW")


engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}/{DATABASE}", echo=False)


Base = declarative_base()


class Accident(Base):
    __tablename__ = "accidents"
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    crash_date = Column(DateTime)
    traffic_control_device = Column(String(100))
    weather_condition = Column(String(100))
    lighting_condition = Column(String(100))
    first_crash_type = Column(String(100))
    trafficway_type = Column(String(100))
    alignment = Column(String(100))
    roadway_surface_cond = Column(String(100))
    road_defect = Column(String(50))
    crash_type = Column(String(100))
    intersection_related_i = Column(String(10))
    damage = Column(String(100))
    prim_contributory_cause = Column(String(255))
    num_units = Column(Integer)
    most_severe_injury = Column(String(100))
    injuries_total = Column(Integer)
    injuries_fatal = Column(Integer)
    injuries_incapacitating = Column(Integer)
    injuries_non_incapacitating = Column(Integer)
    injuries_reported_not_evident = Column(Integer)
    injuries_no_indication = Column(Integer)
    crash_hour = Column(Integer)
    crash_day_of_week = Column(Integer)
    crash_month = Column(Integer)

Base.metadata.create_all(engine)


Session = sessionmaker(bind=engine)
session = Session()

print("Conexión exitosa con la base de datos")

try:
    metadata = MetaData()
    metadata.reflect(bind=engine)
    accidents_table = metadata.tables["accidents"]

    with engine.begin() as conn:
        conn.execute(insert(accidents_table), df.to_dict(orient="records"))

    print("Datos migrados en la tabla 'accidents'")
except Exception as e:
    print(f"Error durante la migración: {e}")



Conexión exitosa con la base de datos
Datos migrados en la tabla 'accidents'
